In [1]:
from scrape import extract_players
from scrape import extract_year
from scrape import make_dists
from scrape import extract_age
from scrape import run_knn
import numpy as np
from sklearn.cluster import KMeans
import pandas as pd
import re
import matplotlib.pyplot as plt
from scrape import read_targets
from scrape import clean_name
from sklearn.svm import SVR
from sklearn.preprocessing import scale
from scrape import svr_2023
from scrape import run_svr_2023
from scrape import run_xgb_2023
from scrape import xgb_2023

In [2]:
results = run_svr_2023('rb', 'PPR', pca=True)
results['projections'].head(50)

austin-ekeler
christian-mccaffrey
josh-jacobs
derrick-henry
saquon-barkley
nick-chubb
rhamondre-stevenson
tony-pollard
aaron-jones
joe-mixon
dalvin-cook
leonard-fournette
jamaal-williams
najee-harris-rb
miles-sanders
alvin-kamara
travis-etienne
kenneth-walker-rb
james-conner
jerick-mckinnon
dandre-swift
ezekiel-elliott
devin-singletary
david-montgomery
raheem-mostert
aj-dillon
dameon-pierce
antonio-gibson
tyler-allgeier
jeffery-wilson
cordarrelle-patterson
latavius-murray
jonathan-taylor
samaje-perine
cam-akers
rachaad-white
isiah-pacheco
donta-foreman
kareem-hunt
michael-carter-rb
khalil-herbert
breece-hall
brian-robinson-jr
james-cook
kenyan-drake
clyde-edwards-helaire
jaylen-warren
damien-harris-rb
eno-benjamin
james-robinson
alexander-mattison
kenneth-gainwell
chuba-hubbard
melvin-gordon
deon-jackson
nyheim-hines
jk-dobbins
dontrell-hilliard
chase-edmonds
darrell-henderson
rex-burkhead
jamycal-hasty
joshua-kelley
zack-moss
matt-breida
gus-edwards
zonovan-bam-knight
jd-mckissic
amee

c:\Python\Python39\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Done for 2022
jonathan-taylor
austin-ekeler
najee-harris-rb
joe-mixon
james-conner
leonard-fournette
ezekiel-elliott
alvin-kamara
cordarrelle-patterson
antonio-gibson
aaron-jones
josh-jacobs
nick-chubb
damien-harris-rb
dandre-swift
dalvin-cook
javonte-williams
devin-singletary
darrel-williams
david-montgomery
melvin-gordon
derrick-henry
aj-dillon
james-robinson
myles-gaskin
elijah-mitchell-rb
darrell-henderson
tony-pollard
michael-carter-rb
saquon-barkley
devonta-freeman
sony-michel
devontae-booker
chase-edmonds
mike-davis-rb
chuba-hubbard
jd-mckissic
christian-mccaffrey
alexander-mattison
brandon-bolden
kenneth-gainwell
rashaad-penny
jamaal-williams
miles-sanders
ty-johnson
clyde-edwards-helaire
rhamondre-stevenson
nyheim-hines
kareem-hunt
mark-ingram
zack-moss
rex-burkhead
dernest-johnson
latavius-murray
kenyan-drake
boston-scott
donta-foreman
ameer-abdullah
justin-jackson-rb
samaje-perine
david-johnson-rb
ronald-jones-ii
khalil-herbert
kyle-juszczyk
jeremy-mcnichols
dontrell-hilliar

c:\Python\Python39\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Done for 2021
alvin-kamara
dalvin-cook
derrick-henry
david-montgomery
aaron-jones
jonathan-taylor
james-robinson
josh-jacobs
ezekiel-elliott
kareem-hunt
nick-chubb
mike-davis-rb
antonio-gibson
melvin-gordon
nyheim-hines
kenyan-drake
jd-mckissic
dandre-swift
chris-carson
ronald-jones-ii
david-johnson-rb
clyde-edwards-helaire
miles-sanders
jk-dobbins
chase-edmonds
austin-ekeler
james-conner
myles-gaskin
todd-gurley
giovani-bernard
devin-singletary
jeffery-wilson
wayne-gallman
latavius-murray
leonard-fournette
darrell-henderson
gus-edwards
jamaal-williams
jerick-mckinnon
adrian-peterson-min
tony-pollard
james-white
malcolm-brown
rex-burkhead
cam-akers
zack-moss
frank-gore
raheem-mostert
joe-mixon
boston-scott
brian-hill
kalen-ballage
damien-harris-rb
christian-mccaffrey
alexander-mattison
devontae-booker
carlos-hyde
duke-johnson
kyle-juszczyk
joshua-kelley
sony-michel
benjamin-snell-jr
leveon-bell
kerryon-johnson
salvon-ahmed
phillip-lindsay
samaje-perine
justin-jackson-rb
ty-johnson
jord

c:\Python\Python39\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Done for 2020
PCA Selected
Train dimensions (pre PCA):  (171, 26)
Test dimensions (pre PCA):  (95, 26)
Train dimensions (post PCA):  (171, 13)
Test dimensions (post PCA):  (95, 13)
Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-6.963 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-5.041 total time=   0.0s
[CV 3/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-6.643 total time=   0.0s
[CV 4/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-5.804 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-4.654 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-6.741 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-4.985 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-6.515 total time=   0.0s
[CV 4/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-5.688 total time=   0.0s
[CV 5/5] END .....C=0.1, gam

,name,proj fpts,class
2,josh-jacobs,15.964412,1
1,christian-mccaffrey,15.845696,0
0,austin-ekeler,15.277109,0
3,derrick-henry,14.292743,1
5,nick-chubb,13.868114,1
4,saquon-barkley,13.820719,1
9,joe-mixon,12.700996,0
15,alvin-kamara,12.616548,0
12,jamaal-williams,12.297742,2
25,dameon-pierce,12.164862,1


In [3]:
dists2022 = results['2022_df']
dists2021 = results['2021_df']
dists2020 = results['2020_df']
proj = xgb_2023('rb', 'PPR', dists2020, dists2021, dists2022)
proj.head(50)

======= Fold 0 ========
Our accuracy on the validation set is 3.8558707230474667
======= Fold 1 ========
Our accuracy on the validation set is 3.956698315321024
======= Fold 2 ========
Our accuracy on the validation set is 3.8585435839629985
======= Fold 3 ========
Our accuracy on the validation set is 4.778674288032001
======= Fold 4 ========
Our accuracy on the validation set is 3.977924678940534
iteration 1: Dimensions = (95, 2)
iteration 2: Dimensions = (190, 2)
iteration 3: Dimensions = (285, 2)
iteration 4: Dimensions = (380, 2)
iteration 5: Dimensions = (475, 2)


,name,proj fpts,class,2022 rank
0,josh-jacobs,16.244777,1,3
1,austin-ekeler,15.470880,0,1
2,christian-mccaffrey,15.462106,0,2
3,derrick-henry,14.388368,1,4
4,nick-chubb,14.013916,1,6
5,saquon-barkley,13.909003,1,5
6,james-conner,13.286069,3,18
7,joe-mixon,13.194524,0,10
8,alvin-kamara,12.712686,0,16
9,dalvin-cook,12.442869,1,11


In [4]:
proj.to_csv('2022_rb_projections_xbr.csv')